https://docs.databricks.com/notebooks/visualizations/index.html

In [0]:
import os.path
fileName = "dbfs:/" + os.path.join('databricks-datasets', 'cs100', 'lab1', 'data-001', 'shakespeare.txt')
print(fileName)
shakespeareRDD = sc.textFile(fileName,8)

dbfs:/databricks-datasets/cs100/lab1/data-001/shakespeare.txt

In [0]:
sc

Out[2]:

SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
shakespeareRDD.take(10)

Out[4]: ['1609',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 ' 1',
 ' From fairest creatures we desire increase,']

In [0]:
shakespeareWordsRDD = shakespeareRDD.map(lambda line: line.split(' '))

In [0]:
shakespeareWordsRDD.count()

Out[6]: 122395

In [0]:
shakespeareWordsRDD.take(5)

Out[8]: [['1609'], [''], ['THE', 'SONNETS'], [''], ['by', 'William', 'Shakespeare']]

In [0]:
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda line: line.split(' '))

In [0]:
shakespeareWordsRDD.take(10)

Out[10]: ['1609', '', 'THE', 'SONNETS', '', 'by', 'William', 'Shakespeare', '', '']

In [0]:
import re
def removeChars(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only whitespace, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    # define punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # remove punctuation from the string
    no_punct = ""
    for char in text:
       if char not in punctuations:
          no_punct = no_punct + char
    return no_punct.lower().strip()

In [0]:
removeChars("Esto es una prueba, FFDDGG[...--´^^^[]], 453634-jhjg")

Out[12]: 'esto es una prueba ffddgg´ 453634jhjg'

In [0]:
shakespeareWordsRDD = shakespeareRDD.map(lambda line: removeChars(line))

In [0]:
shakespeareWordsRDD.take(20)

Out[14]: ['1609',
 '',
 'the sonnets',
 '',
 'by william shakespeare',
 '',
 '',
 '',
 '1',
 'from fairest creatures we desire increase',
 'that thereby beautys rose might never die',
 'but as the riper should by time decease',
 'his tender heir might bear his memory',
 'but thou contracted to thine own bright eyes',
 'feedst thy lights flame with selfsubstantial fuel',
 'making a famine where abundance lies',
 'thy self thy foe to thy sweet self too cruel',
 'thou that art now the worlds fresh ornament',
 'and only herald to the gaudy spring',
 'within thine own bud buriest thy content']

In [0]:
shakespeareWordsRDD_clean = shakespeareWordsRDD.flatMap(lambda line: line.split(' '))
shakespeareWordsRDD_clean.take(10)

Out[15]: ['1609', '', 'the', 'sonnets', '', 'by', 'william', 'shakespeare', '', '']

In [0]:
shakeWordsRDD = shakespeareWordsRDD_clean.filter(lambda w: w.strip()!='')
shakeWordsRDD.take(10)

Out[16]: ['1609',
 'the',
 'sonnets',
 'by',
 'william',
 'shakespeare',
 '1',
 'from',
 'fairest',
 'creatures']

## Lazy Loading

In [0]:
shakespeareRDD_error = shakespeareRDD.map(lambda line: int(line))

In [0]:
shakespeareRDD_error.take(10)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3694539324642075> in <module> 
 ----> 1 shakespeareRDD_error . take ( 10 ) 

 /databricks/spark/python/pyspark/rdd.py in take (self, num) 
 1473 
 1474 p = range ( partsScanned , min ( partsScanned + numPartsToTry , totalParts ) ) 
 -> 1475 res = self . context . runJob ( self , takeUpToNumLeft , p ) 
 1476 
 1477 items += res

 /databricks/spark/python/pyspark/context.py in runJob (self, rdd, partitionFunc, partitions, allowLocal) 
 1225 finally : 
 1226 os . remove ( filename ) 
 -> 1227 sock_info = self . _jvm . PythonRDD . runJob ( self . _jsc . sc ( ) , mappedRDD . _jrdd , partitions ) 
 1228 return list ( _load_from_socket ( sock_info , mappedRDD . _jrdd_deserializer ) ) 
 1229 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 15, ip-10-172-226-103.us-west-2.compute.internal, executor driver): org.apache.spark.api.python.PythonException: 'ValueError: invalid literal for int() with base 10: ''', from <command-3694539324642074>, line 1. Full traceback below:
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 676, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 668, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 279, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/rdd.py", line 1469, in takeUpToNumLeft
 yield next(iterator)
 File "/databricks/spark/python/pyspark/util.py", line 109, in wrapper
 return f(*args, **kwargs)
 File "<command-3694539324642074>", line 1, in <lambda>
ValueError: invalid literal for int() with base 10: ''

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:599)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:734)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:717)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:552)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:105)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:49)
	at scala.collection.TraversableOnce.to(TraversableOnce.scala:315)
	at scala.collection.TraversableOnce.to$(TraversableOnce.scala:313)
	at org.apache.spark.InterruptibleIterator.to(InterruptibleIt

In [0]:
wordCounts_map = shakeWordsRDD.map(lambda w: (w,1))

In [0]:
wordCounts_map.take(10)

Out[20]: [('1609', 1),
 ('the', 1),
 ('sonnets', 1),
 ('by', 1),
 ('william', 1),
 ('shakespeare', 1),
 ('1', 1),
 ('from', 1),
 ('fairest', 1),
 ('creatures', 1)]

In [0]:
wordCounts_map.count()

Out[21]: 882996

In [0]:
wordCounts_map_key = wordCounts_map.reduceByKey(lambda a,b:a+b)
wordCounts_map_key.take(10)

Out[22]: [('thereby', 26),
 ('but', 6265),
 ('tender', 138),
 ('selfsubstantial', 1),
 ('fuel', 5),
 ('where', 1232),
 ('self', 138),
 ('now', 2778),
 ('when', 2049),
 ('besiege', 10)]

In [0]:
top15WordsAndCounts = wordCounts_map_key.takeOrdered(15, key = lambda x: -x[1])

In [0]:
top15WordsAndCounts

Out[24]: [('the', 27361),
 ('and', 26028),
 ('i', 20681),
 ('to', 19150),
 ('of', 17463),
 ('a', 14593),
 ('you', 13615),
 ('my', 12481),
 ('in', 10956),
 ('that', 10890),
 ('is', 9134),
 ('not', 8497),
 ('with', 7771),
 ('me', 7769),
 ('it', 7678)]

In [0]:
def vowel_a_counter(input):
    num_a_vowels= len(re.findall(r'[a+]', input))
    if num_a_vowels>3:
      word= True
    else:
      word= False
    return word


In [0]:
shakeWordsRDD_Condition_True_and_False = wordCounts_map_key.map(lambda x: (x[0],vowel_a_counter(x[0])))
#shakeWordsRDD_Condition_True_and_False = wordCounts_map.map(lambda x: (x[0],vowel_a_counter(x[0])))

In [0]:
shakeWordsRDD_Condition_True_and_False.take(60)

Out[28]: [('thereby', False),
 ('but', False),
 ('tender', False),
 ('selfsubstantial', False),
 ('fuel', False),
 ('where', False),
 ('self', False),
 ('now', False),
 ('when', False),
 ('besiege', False),
 ('of', False),
 ('an', False),
 ('praise', False),
 ('sum', False),
 ('succession', False),
 ('repair', False),
 ('unbless', False),
 ('husbandry', False),
 ('he', False),
 ('4', False),
 ('why', False),
 ('are', False),
 ('free', False),
 ('beauteous', False),
 ('bounteous', False),
 ('profitless', False),
 ('yet', False),
 ('work', False),
 ('frame', False),
 ('fairly', False),
 ('bereft', False),
 ('nor', False),
 ('flowers', False),
 ('let', False),
 ('pay', False),
 ('willing', False),
 ('orient', False),
 ('middle', False),
 ('highmost', False),
 ('weary', False),
 ('like', False),
 ('unlooked', False),
 ('diest', False),
 ('married', False),
 ('pleasing', False),
 ('hast', False),
 ('enjoys', False),
 ('bosom', False),
 ('sits', False),
 ('10', False),
 ('unprovident', False),
 ('gainst', False),
 ('thought', False),
 ('i', False),
 ('presence', False),
 ('would', False),
 ('store', False),
 ('gift', False),
 ('carved', False),
 ('seal', False)]

In [0]:
shakeWordsRDD_Condition_True = shakeWordsRDD_Condition_True_and_False.filter(lambda x:x[1]).map(lambda x:x[0])

In [0]:
shakeWordsRDD_Condition_True.take(10)

Out[32]: ['magistratesalias',
 'advantageable',
 'tragicalcomicalhistoricalpastoral',
 'atalantas']

In [0]:
def vowel_counter(input):
    num_vowels= len(re.findall(r'[aeiou]', input))
    if num_vowels>1 and num_vowels<3:
      word= True
    else:
      word= False
    return word

In [0]:
shakeWordsRDD_Condition_True = shakeWordsRDD.map(lambda x: (x,vowel_counter(x))).filter(lambda x:x[1]).map(lambda x:x[0])

#Filtramos y nos quedamos únicamente con las palabras que cumplen nuestra condición:
shakeWordsRDD_Condition_True.take(10)

Out[36]: ['sonnets',
 'thereby',
 'rose',
 'never',
 'die',
 'riper',
 'should',
 'time',
 'tender',
 'heir']

In [0]:
llista = shakeWordsRDD_Condition_True.collect()

In [0]:
llista



Out[38]: ['sonnets',
 'thereby',
 'rose',
 'never',
 'die',
 'riper',
 'should',
 'time',
 'tender',
 'heir',
 'bear',
 'memory',
 'thou',
 'thine',
 'eyes',
 'feedst',
 'flame',
 'fuel',
 'making',
 'where',
 'lies',
 'foe',
 'sweet',
 'too',
 'cruel',
 'thou',
 'herald',
 'gaudy',
 'within',
 'thine',
 'content',
 'tender',
 'waste',
 'else',
 'glutton',
 'eat',
 'due',
 'grave',
 'thee',
 'winters',
 'deep',
 'trenches',
 'field',
 'youths',
 'proud',
 'livery',
 'gazed',
 'weed',
 'being',
 'asked',
 'where',
 'lies',
 'where',
 'within',
 'thine',
 'deep',
 'sunken',
 'eyes',
 'were',
 'shame',
 'thriftless',
 'more',
 'use',
 'thou',
 'couldst',
 'answer',
 'fair',
 'mine',
 'count',
 'make',
 'proving',
 'thine',
 'were',
 'made',
 'thou',
 'see',
 'blood',
 'thou',
 'feelst',
 'look',
 'face',
 'thou',
 'time',
 'face',
 'should',
 'whose',
 'thou',
 'thou',
 'unbless',
 'some',
 'mother',
 'where',
 'fair',
 'whose',
 'husbandry',
 'thou',
 'mothers',
 'thee',
 'lovely',
 'april',
 'prime',
 'thou',
 'through',
 'windows',
 'thine',
 'age',
 'see',
 'wrinkles',
 'golden',
 'time',
 'thou',
 'live',
 'die',
 'single',
 'thine',
 'dies',
 'thee',
 'unthrifty',
 'thou',
 'upon',
 'legacy',
 'gives',
 'nothing',
 'being',
 'those',
 'are',
 'free',
 'niggard',
 'thou',
 'largess',
 'given',
 'thee',
 'give',
 'thou',
 'use',
 'great',
 'live',
 'having',
 'traffic',
 'thou',
 'sweet',
 'thee',
 'gone',
 'thou',
 'tombed',
 'thee',
 'used',
 'lives',
 'those',
 'hours',
 'gentle',
 'frame',
 'lovely',
 'gaze',
 'where',
 'every',
 'eye',
 'same',
 'fairly',
 'excel',
 'time',
 'leads',
 'summer',
 'winter',
 'there',
 'checked',
 'gone',
 'every',
 'where',
 'were',
 'summers',
 'effect',
 'were',
 'bereft',
 'flowers',
 'though',
 'winter',
 'meet',
 'their',
 'their',
 'lives',
 'sweet',
 'winters',
 'ragged',
 'thee',
 'summer',
 'ere',
 'thou',
 'make',
 'sweet',
 'some',
 'vial',
 'thou',
 'some',
 'place',
 'ere',
 'use',
 'usury',
 'those',
 'willing',
 'loan',
 'breed',
 'thee',
 'times',
 'one',
 'times',
 'were',
 'thou',
 'thine',
 'times',
 'thee',
 'could',
 'death',
 'thou',
 'shouldst',
 'depart',
 'thee',
 'living',
 'thou',
 'too',
 'fair',
 'deaths',
 'make',
 'thine',
 'heir',
 'burning',
 'head',
 'each',
 'under',
 'eye',
 'serving',
 'looks',
 'sacred',
 'majesty',
 'having',
 'climbed',
 'youth',
 'middle',
 'age',
 'mortal',
 'looks',
 'golden',
 'highmost',
 'weary',
 'like',
 'age',
 'eyes',
 'fore',
 'are',
 'look',
 'thou',
 'noon',
 'diest',
 'unless',
 'thou',
 'music',
 'hear',
 'hearst',
 'thou',
 'music',
 'sweets',
 'sweets',
 'delights',
 'thou',
 'thou',
 'else',
 'thine',
 'annoy',
 'true',
 'concord',
 'sounds',
 'offend',
 'thine',
 'ear',
 'sweetly',
 'chide',
 'thee',
 'thou',
 'shouldst',
 'bear',
 'one',
 'sweet',
 'husband',
 'strikes',
 'each',
 'each',
 'sire',
 'mother',
 'one',
 'one',
 'note',
 'whose',
 'being',
 'one',
 'thee',
 'thou',
 'single',
 'prove',
 'none',
 'fear',
 'widows',
 'eye',
 'thou',
 'consumst',
 'single',
 'life',
 'thou',
 'die',
 'wail',
 'thee',
 'like',
 'wife',
 'widow',
 'weep',
 'thou',
 'thee',
 'behind',
 'every',
 'widow',
 'keep',
 'childrens',
 'eyes',
 'husbands',
 'shape',
 'look',
 'unthrift',
 'place',
 'enjoys',
 'waste',
 'user',
 'destroys',
 'love',
 'toward',
 'others',
 'bosom',
 'himself',
 'shame',
 'commits',
 'shame',
 'thou',
 'bearst',
 'love',
 'thou',
 'thou',
 'thou',
 'none',
 'thou',
 'hate',
 'gainst',
 'thou',
 'roof',
 'should',
 'chief',
 'change',
 'thought',
 'change',
 'hate',
 'lodged',
 'gentle',
 'love',
 'least',
 'prove',
 'make',
 'thee',
 'love',
 'live',
 'thine',
 'thee',
 'thou',
 'wane',
 'thou',
 'one',
 'thine',
 'thou',
 'blood',
 'youngly',
 'thou',
 'bestowst',
 'thou',
 'thine',
 'thou',
 'youth',
 'lives',
 'wisdom',
 'age',
 'decay',
 'were',
 'minded',
 'times',
 'should',
 'year',
 'would',
 'make',
 'away',
 'those',
 'made',
 'store',
 'rude',
 'barrenly',
 'perish',
 'look',
 'gave',
 'the

In [0]:
len(llista)

Out[39]: 270493